## Effective Python
Learning coding motifs.



In [12]:
# Know how closures interact with variable scope... 

'''
Say you want to prioritize one group of numbers to come first,
this happens when rendering a user interface you want 
important msgs or exceptional events to be displayed first.

A common way to do this is to pass a helper function as 
the key argument to a list's sort method... the helper's return
value will be used as the value for sorting each item in the list

1) Python supports closures
 - notice how helper(x) can access "group" 

2) Functions are first-class objects in Python
 - so you can refer to them directly... 
 - and also pass them as arguments to other functions
 - this is why values.sort() can use "key=helper"
 - helper is a function! 
'''

def sort_priority(values, group): 
    def helper(x): 
        if x in group: 
            return (0,x) 
        return (1,x) 
    values.sort(key=helper)

In [8]:
numbers = [8,3,1,2,5,4,7,6] # notice list
group = {2,3,5,7} # notice set 
sort_priority(numbers, group)
print(numbers) 

[2, 3, 5, 7, 1, 4, 6, 8]


In [15]:
'''
The sorted results are correct, but the found result is wrong.
Items from group were definitely found in numbers, but the 
function returned False... 

When you reference a variable in an expression, the Python
interpreter will traverse the scope to resolve the reference
in this order: 

1) the current function's scope
2) any enclosing scopes (like other containing functions) 
3) the scope of the module (global scope) 
4) the built-in scope (that contains functions like len and str)

If none of these places have a defined variable then a 
NameError exception is raised... 


So....

What's happening is that found = False in scope of sort_priority2
The closure's assignment is treated as a NEW variable definition
within helper... NOT as an ASSIGNMENT within sort_priority2!

THIS IS A SCOPING BUG. This is totally intended. 
Prevents local variables in a function from polluting the 
containing module. 
'''

def sort_priority2(numbers, group): 
    found = False # Scope: 'sort_priority2'
    def helper(x): 
        if x in group: 
            found = True # seems simple, but wrong!
                         # Scope: 'helper' -- bad! 
            return (0,x)
        return (1,x) 
    numbers.sort(key=helper) 
    return found



In [17]:
found = sort_priority2(numbers, group)
print('Found:', found) 
print(numbers) 

# so you can see that Found: False is wrong... 
# this is a problem of scope! 

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


In [21]:
'''
This is how you "get data out" ... you need to evoke a special 
syntax to get data out of a closure. The keyword is "nonlocal" 

nonlocal => scope traversal should happen upon assignment for 
a specific variable name... nonlocal will NOT traverse up to the 
module-level scope (to avoid polluting globals) 

Caution: probably shouldn't use nonlocal for anything beyond 
simple functions. Side effects are hard to follow. Instead, use
helper classes... 

'''

def sort_priority3(numbers, group):
    found = False
    def helper(x): 
        nonlocal found # key difference! 
        if x in group: 
            found = True
            return (0,x) 
        return (1,x) 
    numbers.sort(key=helper) 
    return found


In [22]:
'''
This class achieves the same result as the nonlocal approach.
# It's best to avoid the use of nonlocal outside of simple 
functions... because the side effects can be hard to spot. 
'''

class Sorter(object): 
    def __init__(self, group): 
        self.group = group
        self.found = False
        
    def __call__(self, x): 
        if x in self.group: 
            self.found = True
            return(0,x)
        return (1,x) 
    
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True